# 3. SQL para preparación de datos

# 3.1 Introducción

En capítulos anteriores, adquiriste una comprensión fundamental sobre análisis de datos y SQL. Dominaste cómo utilizar las instrucciones CREATE, INSERT, SELECT, ALTER, UPDATE, DELETE y DROP en SQL para desplegar operaciones de creación, lectura, actualización y eliminación (conocidas como CRUD por sus siglas en inglés). Estos conceptos son esenciales para el análisis de datos.

No obstante, en la realidad profesional, como analista de datos, raramente gestionarás todo el proceso CRUD de inicio a fin. Comúnmente, no generas conjuntos de datos desde cero, sino que los recibes de fuentes externas. A menudo, estos datos presentan formatos que requieren adaptaciones antes de ser utilizables. Puede que falte información, que algunos datos no se ajusten al formato requerido, o incluso que haya información incorrecta.

Forbes estima que cerca del 80% del tiempo empleado por los analistas se dedica a la preparación de datos. Usar datos impuros para construir modelos puede comprometer los análisis, llevando a conclusiones inadecuadas. Aquí es donde SQL destaca, ofreciendo herramientas eficientes para desarrollar conjuntos de datos limpios y estructurados.

En este capítulo, exploraremos cómo consolidar datos usando JOIN y UNION. También abordaremos funciones como CASE WHEN, COALESCE, NULLIF y LEAST/GREATEST para la limpieza de datos. Finalmente, aprenderás a transformar y eliminar datos duplicados mediante el comando DISTINCT.

# 3.2 Ensamble de Datos

En el segundo capítulo, titulado "Fundamentos de SQL para Análisis", aprendiste operaciones básicas con una sola tabla. Pero, ¿qué sucede si la información requerida se distribuye en varias tablas? En esta sección, descubrirás cómo consolidar datos de múltiples tablas mediante uniones y agrupaciones.

## 3.2.1  Conexión de Tablas con JOIN


En muchas ocasiones, la información deseada reside en diversas tablas. Un simple comando SELECT sobre una tabla no basta. Sin embargo, SQL nos brinda herramientas para integrar tablas relacionadas a través del término JOIN.

Considera, por ejemplo, dos tablas en la base de datos ZoomZoom: una de concesionarios y otra de vendedores.

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/29104fd3-90f7-4272-8ff5-8b8c10991883.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

En la tabla de vendedores, puedes observar que hay una columna llamada dealership_id. Esta columna dealership_id es una referencia directa a la columna dealership_id en la tabla de concesionarios. Cuando la tabla A tiene una columna que referencia la clave principal de la tabla B, se dice que la columna es una clave extranjera para la tabla A. En este caso, la columna dealership_id en vendedores es una clave extranjera para la tabla de concesionarios.

Dado que estas dos tablas están relacionadas, puedes realizar algunos análisis interesantes con ellas. Por ejemplo, puedes estar interesado en determinar qué vendedores trabajan en un concesionario en California. Una forma de obtener esta información es primero consultar cuáles concesionarios están en California. Puedes hacerlo usando la siguiente consulta:

```sql
SELECT *
FROM dealerships
WHERE state='CA';
```

Esta consulta debería darte los siguientes resultados:


<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/6ae4f3fd-9db3-4a3e-bf0b-7cfd759aa218.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

Figura 3.3: Concesionarios en California

Ahora que sabes que los únicos dos concesionarios en California tienen los IDs 2 y 5, respectivamente, puedes entonces consultar la tabla de vendedores, de la siguiente manera:

```sql
SELECT *
FROM salespeople
WHERE dealership_id in (2, 5)
ORDER BY 1;
```

A continuación se muestran las primeras nueve filas del resultado del código:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/41556d8f-3e2c-4510-b891-88f926d7b022.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

Figura 3.4: Vendedores en California

Aunque este método te da los resultados que quieres, es tedioso realizar dos consultas para obtener estos resultados. Lo que haría este proceso más fácil sería de alguna manera añadir la información de la tabla de concesionarios a la tabla de vendedores y luego filtrar por los usuarios en California. SQL proporciona tal herramienta con la cláusula JOIN. La cláusula JOIN es una cláusula SQL que permite a un usuario unir una o más tablas juntas basándose en condiciones distintas.

### Tipos de Joins
En este capítulo, aprenderás sobre tres joins fundamentales, que se ilustran en la siguiente figura: joins internos, joins externos y cross joins:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images2b02b3cf-cf94-43f7-96c8-de59d23ba3f7.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>


Figura 3.5: Tipos principales de joins

#### Joins Internos

Un join interno conecta filas en diferentes tablas, basándose en una condición conocida como el predicado de join. En muchos casos, el predicado de join es una condición lógica de igualdad. Cada fila en la primera tabla se compara con cada otra fila en la segunda tabla. Para las combinaciones de filas que cumplen el predicado de join interno, esa fila se devuelve en la consulta. De lo contrario, la combinación de filas se descarta.

Los joins internos generalmente se escriben de la siguiente forma:

```sql
SELECT {columns}
FROM {table1}
INNER JOIN {table2}
  ON {table1}.{common_key_1}={table2}.{common_key_2};
```

Aquí, {columns} son las columnas que quieres obtener de la tabla unida, {table1} es la primera tabla, {table2} es la segunda tabla, {common_key_1} es la columna en {table1} a la que quieres unirte, y {common_key_2} es la columna en {table2} a la que quieres unirte.

Ahora, regresa a las dos tablas discutidas anteriormente: concesionarios y vendedores. Como se mencionó anteriormente, sería bueno si pudieras agregar la información de la tabla de concesionarios a la tabla de vendedores sabiendo en qué estado está cada concesionario. Por el momento, supone que todos los ID de los vendedores tienen un valor dealership_id válido.

Puedes unir las dos tablas usando una condición de igualdad en el predicado de unión, como sigue:

```sql
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
ORDER BY 1;
```

La siguiente figura muestra las primeras filas del resultado:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/7b05c1b2-f698-4eea-8593-cfebb8519a76.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

Figura 3.6: La tabla de vendedores unida a la tabla de concesionarios

Como puedes ver en el resultado anterior, la tabla es el resultado de unir la tabla de vendedores con la de concesionarios. Nota que la primera tabla listada en la consulta, vendedores, está en el lado izquierdo del resultado, mientras que la tabla de concesionarios está en el lado derecho. Este orden de izquierda-derecha será muy importante en la próxima sección cuando aprendas sobre los joins externos entre tablas. Durante un join externo, si una tabla está en el lado izquierdo o derecho puede afectar el resultado de la consulta. Para un join interno, sin embargo, el orden de las tablas no es importante para los predicados de unión que usan una operación de igualdad.

Ahora, mira las columnas involucradas; dealership_id en la tabla de vendedores coincide con dealership_id en la tabla de concesionarios. Esto muestra cómo se cumple el predicado de unión. Al ejecutar esta consulta de unión, has creado efectivamente un nuevo "superconjunto de datos" que consta de las dos tablas fusionadas donde las dos columnas dealership_id son iguales.

Ahora puedes ejecutar una consulta SELECT sobre este "superconjunto de datos" de la misma manera que una tabla grande usando las cláusulas y palabras clave del Capítulo 2, Los conceptos básicos de SQL para análisis. Por ejemplo, volviendo al problema de múltiples consultas para determinar cuál de las consultas de ventas funciona en California, ahora puedes abordarlo con una consulta fácil:

```sql
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1;
```

Esto te da el siguiente resultado, que muestra las primeras filas del conjunto de resultados completo:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/5ffdefff-aee9-4e68-ae12-170d1844cc13.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

Figura 3.7: Vendedores en California con una consulta

Observarás que el resultado en la Figura 3.6 y la Figura 3.7 es casi idéntico, siendo la excepción que la tabla en la Figura 3.7 tiene los datos de los concesionarios añadidos también. Si quieres recuperar solo la parte de la tabla de vendedores de esto, puedes seleccionar las columnas de vendedores usando la siguiente sintaxis de estrella:

```sql
SELECT salespeople.*
FROM salespeople
INNER JOIN dealerships
  ON dealerships.dealership_id = salespeople.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1;
```

Aquí están las primeras filas devueltas por esta consulta:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/dba76af2-bf81-41c2-a641-d77fae772aa9.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>

Figura 3.8: Vendedores en California con SELECT alias de tabla

Hay otro atajo que puede ayudar mientras


<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/dba76af2-bf81-41c2-a641-d77fae772aa9.png?raw=true' width="300" />
<figcaption>
Figure 3.1: Structure of dealerships table".</figcaption></center>
</figure>